In [56]:
import pandas as pd
import os
# Loading each dataset manually
data1 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"))
data2 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"))
data3 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Friday-WorkingHours-Morning.pcap_ISCX.csv"))
data4 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Monday-WorkingHours.pcap_ISCX.csv"))
data5 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"))
data6 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"))
data7 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Tuesday-WorkingHours.pcap_ISCX.csv"))
data8 = pd.read_csv(os.path.expanduser("/Users/praneethmuthyalapati/Desktop/DATASET/Wednesday-workingHours.pcap_ISCX.csv"))

# Combine all dataframes into one
data_list = [data1, data2, data3, data4, data5, data6, data7, data8]
df = pd.concat(data_list, ignore_index=True)
print(df.columns.tolist())


[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag 

In [58]:
import numpy as np
df.columns = df.columns.str.strip()
features = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
            'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
            'Flow Bytes/s', 'Fwd Packets/s', 'Packet Length Std']
X = df[features].fillna(0)

# Remove infinite values
X.replace([np.inf, -np.inf], 0, inplace=True)
df['Label'] = df['Label'].astype(str).apply(lambda x: 0 if 'Benign' in x else 1)
y = df['Label']



In [60]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Assume X and y are already defined above this
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, predictions) * 100:.2f}%")

joblib.dump(model, 'intrusion_detection_model.pkl')




Model Accuracy: 100.00%


['intrusion_detection_model.pkl']

In [61]:
import joblib

# Load the pre-trained model
model = joblib.load('intrusion_detection_model.pkl')


In [76]:

import scapy.all as scapy
import numpy as np

# Initialize flow variables globally
flow_start_time = None
flow_end_time = None
flow_source_ip = None
flow_dest_ip = None
fwd_packets = 0
bwd_packets = 0
fwd_pkt_len = []
bwd_pkt_len = []
flow_bytes = 0
flow_packets = []

def extract_features(packet):
    global flow_start_time, flow_end_time, flow_source_ip, flow_dest_ip
    global fwd_packets, bwd_packets, fwd_pkt_len, bwd_pkt_len, flow_bytes, flow_packets

    try:
        # Only process IP packets
        if packet.haslayer(scapy.IP):
            print(f"Processing packet: {packet[scapy.IP].src} -> {packet[scapy.IP].dst}")  # Debugging print

            # Set source and destination IP for the flow (if not already set)
            if flow_source_ip is None and flow_dest_ip is None:
                flow_source_ip = packet[scapy.IP].src
                flow_dest_ip = packet[scapy.IP].dst
                print(f"Flow started: {flow_source_ip} -> {flow_dest_ip}")  # Debugging print

            # Track flow duration
            if flow_start_time is None:
                flow_start_time = packet.time
                print(f"Flow start time: {flow_start_time}")  # Debugging print
            flow_end_time = packet.time

            # Count forward and backward packets and track their lengths
            if packet[scapy.IP].src == flow_source_ip and packet[scapy.IP].dst == flow_dest_ip:
                fwd_packets += 1
                fwd_pkt_len.append(len(packet))
                print(f"Forward packet: {len(packet)} bytes")  # Debugging print
            elif packet[scapy.IP].src == flow_dest_ip and packet[scapy.IP].dst == flow_source_ip:
                bwd_packets += 1
                bwd_pkt_len.append(len(packet))
                print(f"Backward packet: {len(packet)} bytes")  # Debugging print

            # Count total flow bytes
            flow_bytes += len(packet)

            # Store the packet for further analysis (optional)
            flow_packets.append(packet)

            # Calculate features when flow duration is greater than 0
            if flow_end_time - flow_start_time > 0:  # flow duration is non-zero
                flow_duration = flow_end_time - flow_start_time
                fwd_pkt_len_mean = np.mean(fwd_pkt_len) if fwd_pkt_len else 0
                bwd_pkt_len_mean = np.mean(bwd_pkt_len) if bwd_pkt_len else 0
                flow_bytes_per_second = flow_bytes / flow_duration if flow_duration > 0 else 0
                fwd_packets_per_second = fwd_packets / flow_duration if flow_duration > 0 else 0
                pkt_len_std = np.std([len(pkt) for pkt in flow_packets]) if flow_packets else 0

                # Debugging print to check the extracted features
                print(f"Extracted features: [flow_duration: {flow_duration}, "
                      f"fwd_packets: {fwd_packets}, bwd_packets: {bwd_packets}, "
                      f"fwd_pkt_len_mean: {fwd_pkt_len_mean}, bwd_pkt_len_mean: {bwd_pkt_len_mean}, "
                      f"flow_bytes_per_second: {flow_bytes_per_second}, "
                      f"fwd_packets_per_second: {fwd_packets_per_second}, pkt_len_std: {pkt_len_std}]")

                return [flow_duration, fwd_packets, bwd_packets, fwd_pkt_len_mean, bwd_pkt_len_mean,
                        flow_bytes_per_second, fwd_packets_per_second, pkt_len_std]
            else:
                print("Flow duration is zero or negative, skipping feature extraction.")  # Debugging print
        else:
            print("Packet is not an IP packet.")  # Debugging print
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None

# Example function to call extract_features and detect intrusion
def packet_callback(packet):
    features = extract_features(packet)
    if features:
        print(f"Features extracted: {features}")  # Debugging print
    else:
        print("No features extracted.")  # Debugging print

scapy.sniff(prn=detect_intrusion, store=0, count=100)  # Stops after 100 packets



Packet is not an IP packet.
Packet is not an IP packet.
Packet is not an IP packet.
Processing packet: 192.168.1.20 -> 192.168.1.255
Flow started: 192.168.1.20 -> 192.168.1.255
Flow start time: 1743737060.828785
Forward packet: 243 bytes
Flow duration is zero or negative, skipping feature extraction.
Processing packet: 192.168.1.38 -> 142.251.186.188
Extracted features: [flow_duration: 0.924605131149292, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 243.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 321.21820439264326, fwd_packets_per_second: 1.0815427757328056, pkt_len_std: 94.5]
Intrusion detected!
Processing packet: 192.168.1.38 -> 142.250.113.188
Extracted features: [flow_duration: 0.9246959686279297, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 243.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 379.58422217500987, fwd_packets_per_second: 1.0814365304131335, pkt_len_std: 89.09545442950498]
Intrusion detected!
Processing packet: 142.251.186.188 -> 192.168.1.38
Extracted fea

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [83]:
intrusion_count = 0  # Track the number of intrusions

def detect_intrusion(packet):
    global intrusion_count
    max_intrusions = 20  # Stop after detecting 3 intrusions

    # Continue sniffing if we haven't reached the maximum intrusion threshold
    if intrusion_count >= max_intrusions:
        return True  # Return True to stop sniffing

    features = extract_features(packet)
    if features:
        # Convert features to DataFrame for the model
        feature_df = pd.DataFrame([features], columns=['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
                                                      'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
                                                      'Flow Bytes/s', 'Fwd Packets/s', 'Packet Length Std'])
        
        # Fill missing values with 0 and handle infinite values
        feature_df = feature_df.fillna(0)
        feature_df.replace([np.inf, -np.inf], 0, inplace=True)
        
        # Predict using the model
        prediction = model.predict(feature_df)
        
        # Print result (0 for benign, 1 for attack)
        if prediction == 1:
            intrusion_count += 1  # Increment intrusion count
            print(f"Intrusion detected! Total intrusions detected: {intrusion_count}")
            
            # Stop sniffing if we have detected the maximum number of intrusions
            if intrusion_count >= max_intrusions:
                print("Max intrusions detected, stopping sniffing.")
                return True  # Return True to stop sniffing
        else:
            print("Benign traffic detected.")
    
    return False  # Continue sniffing if not reached max intrusions

# Start sniffing and detect intrusions, stop if the specified number of intrusions is detected
scapy.sniff(prn=detect_intrusion, store=0, stop_filter=detect_intrusion)


Packet is not an IP packet.
False
Packet is not an IP packet.
Packet is not an IP packet.
False
Packet is not an IP packet.
Processing packet: 192.168.1.38 -> 142.251.186.105
Extracted features: [flow_duration: 395.23051500320435, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 243.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 117.87298356662136, fwd_packets_per_second: 0.0025301690078052106, pkt_len_std: 495.1115526208603]
Intrusion detected! Total intrusions detected: 1
False
Processing packet: 192.168.1.38 -> 142.251.186.105
Extracted features: [flow_duration: 395.23051500320435, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 243.0, bwd_pkt_len_mean: 0, flow_bytes_per_second: 121.14196192470568, fwd_packets_per_second: 0.0025301690078052106, pkt_len_std: 499.46291070560534]
Intrusion detected! Total intrusions detected: 2
Processing packet: 192.168.1.38 -> 142.251.186.105
Extracted features: [flow_duration: 395.23057413101196, fwd_packets: 1, bwd_packets: 0, fwd_pkt_len_mean: 

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [91]:
!pip install ipywidgets


In [93]:
!jupyter notebook extension enable --py widgetsnbextension


Extension package aext_assistant took 1.0682s to import
****************** ENVIRONMENT [Panels] Environment.PRODUCTION ******************
****************** ENVIRONMENT [Panels] Environment.PRODUCTION ******************
--> Alembic Config: {'db_conn_url': 'sqlite:////Users/praneethmuthyalapati/anaconda_projects/db/project_filebrowser.db', 'db_path': '/Users/praneethmuthyalapati/anaconda_projects/db/project_filebrowser.db'}
Default dir: /Users/praneethmuthyalapati. Running chdir...
Checking for database file @ /Users/praneethmuthyalapati/anaconda_projects/db/project_filebrowser.db
Database file already exists
Migrations executed
Checking project configuration: /Users/praneethmuthyalapati. Running chdir...
Extension package aext_toolbox took 0.1853s to import
A `_jupyter_server_extension_points` function was not found in jupyter_lsp. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyte

In [97]:
import ipywidgets as widgets
from IPython.display import display

# Create a simple button widget
button = widgets.Button(description="Click Me!")
# Define a function to handle the button click event
def on_button_click(b):
    print("Button clicked!")
    
button.on_click(on_button_click)

# Display the button
display(button)


Button(description='Click Me!', style=ButtonStyle())

In [95]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import joblib
import numpy as np

# Load pre-trained model
model = joblib.load('intrusion_detection_model.pkl')

# Create widgets for UI
flow_duration_input = widgets.FloatText(
    value=10.0,
    description='Flow Duration:',
    disabled=False
)

fwd_packets_input = widgets.IntText(
    value=100,
    description='Fwd Packets:',
    disabled=False
)

bwd_packets_input = widgets.IntText(
    value=50,
    description='Bwd Packets:',
    disabled=False
)

fwd_pkt_len_input = widgets.FloatText(
    value=80.0,
    description='Fwd Pkt Length Mean:',
    disabled=False
)

bwd_pkt_len_input = widgets.FloatText(
    value=40.0,
    description='Bwd Pkt Length Mean:',
    disabled=False
)

flow_bytes_input = widgets.FloatText(
    value=2000.0,
    description='Flow Bytes/s:',
    disabled=False
)

fwd_packets_per_sec_input = widgets.FloatText(
    value=10.0,
    description='Fwd Pkts/s:',
    disabled=False
)

pkt_len_std_input = widgets.FloatText(
    value=20.0,
    description='Pkt Length Std:',
    disabled=False
)

predict_button = widgets.Button(description="Predict Intrusion")

# Output widget for displaying results
output = widgets.Output()

# Function to handle button click event
def on_predict_button_click(b):
    # Get input values
    flow_duration = flow_duration_input.value
    fwd_packets = fwd_packets_input.value
    bwd_packets = bwd_packets_input.value
    fwd_pkt_len = fwd_pkt_len_input.value
    bwd_pkt_len = bwd_pkt_len_input.value
    flow_bytes = flow_bytes_input.value
    fwd_packets_per_sec = fwd_packets_per_sec_input.value
    pkt_len_std = pkt_len_std_input.value
    
    # Prepare the feature vector
    features = np.array([flow_duration, fwd_packets, bwd_packets, fwd_pkt_len, bwd_pkt_len, flow_bytes,
                         fwd_packets_per_sec, pkt_len_std]).reshape(1, -1)
    
    # Predict using the pre-trained model
    prediction = model.predict(features)
    
    with output:
        # Display the result (1 for intrusion, 0 for benign)
        if prediction == 1:
            print("Intrusion Detected!")
        else:
            print("Benign Traffic")

# Link button click to function
predict_button.on_click(on_predict_button_click)

# Display widgets
display(flow_duration_input, fwd_packets_input, bwd_packets_input, fwd_pkt_len_input, 
        bwd_pkt_len_input, flow_bytes_input, fwd_packets_per_sec_input, pkt_len_std_input, 
        predict_button, output)


FloatText(value=10.0, description='Flow Duration:')

IntText(value=100, description='Fwd Packets:')

IntText(value=50, description='Bwd Packets:')

FloatText(value=80.0, description='Fwd Pkt Length Mean:')

FloatText(value=40.0, description='Bwd Pkt Length Mean:')

FloatText(value=2000.0, description='Flow Bytes/s:')

FloatText(value=10.0, description='Fwd Pkts/s:')

FloatText(value=20.0, description='Pkt Length Std:')

Button(description='Predict Intrusion', style=ButtonStyle())

Output()